In [52]:
import sqlite3
import pandas as pd
import os
import pandasql
import json 
from collections import deque

In [53]:
conn = sqlite3.connect("readychef.db")
cursor = conn.cursor()

In [54]:
def execute(query, database):
    database.execute(query)
    database.commit()

In [55]:
def allrowsSelect(conn,query):
    cursor = conn.execute(query)
    for row in cursor:
        print(row) 

In [56]:
def execSQL(conn,query):
    conn.execute(query) # execute an SQL query
    conn.commit() # "commit" that query in order to make its action permanent

In [57]:
#q1
bio=pd.read_json('bios.json')
bio



,_id,name,birth,death,contribs,awards,title
0,1,"{'first': 'John', 'last': 'Backus'}",1924-12-03,2007-03-17,"[Fortran, ALGOL, Backus-Naur Form, FP]","[{'award': 'W.W. McDowell Award', 'year': 1967...",NaN
1,2,"{'first': 'John', 'last': 'McCarthy'}",1927-09-04,2011-12-24,"[Lisp, Artificial Intelligence, ALGOL]","[{'award': 'Turing Award', 'year': 1971, 'by':...",NaN
2,3,"{'first': 'Grace', 'last': 'Hopper'}",1906-12-09,1992-01-01,"[UNIVAC, compiler, FLOW-MATIC, COBOL]",[{'award': 'Computer Sciences Man of the Year'...,Rear Admiral
3,4,"{'first': 'Kristen', 'last': 'Nygaard'}",1926-08-27,2002-08-10,"[OOP, Simula]","[{'award': 'Rosing Prize', 'year': 1999, 'by':...",NaN
4,5,"{'first': 'Ole-Johan', 'last': 'Dahl'}",1931-10-12,2002-06-29,"[OOP, Simula]","[{'award': 'Rosing Prize', 'year': 1999, 'by':...",NaN
5,6,"{'first': 'Guido', 'last': 'van Rossum'}",1956-01-31,NaN,[Python],[{'award': 'Award for the Advancement of Free ...,NaN
6,7,"{'first': 'Dennis', 'last': 'Ritchie'}",1941-09-09,2011-10-12,"[UNIX, C]","[{'award': 'Turing Award', 'year': 1983, 'by':...",NaN
7,8,"{'first': 'Yukihiro', 'aka': 'Matz', 'last': '...",1965-04-14,NaN,[Ruby],[{'award': 'Award for the Advancement of Free ...,NaN
8,9,"{'first': 'James', 'last': 'Gosling'}",1955-05-19,NaN,[Java],"[{'award': 'The Economist Innovation Award', '...",NaN
9,10,"{'first': 'Martin', 'last': 'Odersky'}",NaN,NaN,[Scala],NaN,NaN


In [58]:
#The main table  computer_scientists should contain id, last, first, birth, death
# second table named  contribs id, contribution
# third table id, awards 
#forth table named duration, id, time 
# fivth 


In [59]:
awards = bio[['_id','awards']]
awards = awards.explode('awards', ignore_index=True)
awards.head()

,_id,awards
0,1,"{'award': 'W.W. McDowell Award', 'year': 1967,..."
1,1,"{'award': 'National Medal of Science', 'year':..."
2,1,"{'award': 'Turing Award', 'year': 1977, 'by': ..."
3,1,"{'award': 'Draper Prize', 'year': 1993, 'by': ..."
4,2,"{'award': 'Turing Award', 'year': 1971, 'by': ..."


In [60]:
query='''

DROP TABLE IF EXISTS  computerscientists
'''
execSQL(conn,query)

In [61]:
Table1='''

CREATE TABLE computerscientists (
 "id" INTEGER,
"first"	TEXT,
"last"	TEXT,
"aka"	TEXT,
"birth"	TEXT,
"death"	TEXT,
PRIMARY KEY("id")
);
'''

execSQL(conn,Table1)

In [62]:
query='''

DROP TABLE IF EXISTS  awards
'''
execSQL(conn,query)

In [63]:
Table2='''
CREATE TABLE awards (
       id integer primary key,
    award text,
    year text,
    by text
 
);
'''

execSQL(conn,Table2)

In [64]:
db = sqlite3.connect('Bio.db')

In [65]:
execute("PRAGMA foreign_keys=ON", db)
cursor=db.cursor()

In [66]:
bio['awards'][0]
awards = bio[['_id','awards']]
awards = awards.explode('awards', ignore_index=True).dropna()
awards.head()


,_id,awards
0,1,"{'award': 'W.W. McDowell Award', 'year': 1967,..."
1,1,"{'award': 'National Medal of Science', 'year':..."
2,1,"{'award': 'Turing Award', 'year': 1977, 'by': ..."
3,1,"{'award': 'Draper Prize', 'year': 1993, 'by': ..."
4,2,"{'award': 'Turing Award', 'year': 1971, 'by': ..."


In [67]:
contributions = bio[['_id','contribs']]
contributions = contributions.explode('contribs', ignore_index=True)
contributions.head()

,_id,contribs
0,1,Fortran
1,1,ALGOL
2,1,Backus-Naur Form
3,1,FP
4,2,Lisp


In [68]:
computerscientists = bio['name']
computerscientists.head()
computerscientists = pd.json_normalize(computerscientists)
computerscientists[['_id','birth','death']] = bio[['_id','birth','death']]
computerscientists.head()

,first,last,aka,_id,birth,death
0,John,Backus,NaN,1,1924-12-03,2007-03-17
1,John,McCarthy,NaN,2,1927-09-04,2011-12-24
2,Grace,Hopper,NaN,3,1906-12-09,1992-01-01
3,Kristen,Nygaard,NaN,4,1926-08-27,2002-08-10
4,Ole-Johan,Dahl,NaN,5,1931-10-12,2002-06-29


In [69]:
title = bio[['_id','title']]
duration=bio[['_id','birth','death']]

In [70]:
query='''

DROP TABLE IF EXISTS  duration
'''
execSQL(conn,query)

In [71]:
Table3='''
CREATE TABLE duration (
 "id" INTEGER,
 "birth" TEXT,
 "death" TEXT
);
'''

execSQL(conn,Table3)

In [72]:
query='''

DROP TABLE IF EXISTS  contributions
'''
execSQL(conn,query)

In [73]:
Table4='''
CREATE TABLE contributions (
   id integer primary key,
    contribution text
	
);
'''

execSQL(conn,Table4)

In [74]:
query='''

DROP TABLE IF EXISTS  title
'''
execSQL(conn,query)

In [75]:
Table5='''
CREATE TABLE title(
    id integer ,
    title text,
   FOREIGN KEY (id) REFERENCES computerscientists(id)
  
);
'''

execSQL(conn,Table5)

In [76]:
import csv
cursor = conn.cursor()
title.to_csv('title.csv', index = False)
duration.to_csv('duration.csv', index = False)

In [77]:
contributions.to_csv('contributions.csv', index = False)
computerscientists.to_csv('computerscientists.csv', index = False)
awards.to_csv('awards.csv', index = False)

In [78]:
file = open('title.csv')

c = csv.reader(file)
next(c)

n = "INSERT INTO title(id, title) VALUES(?, ?)"

cursor.executemany(n, c)

select_all = "SELECT * FROM title"
rows = cursor.execute(select_all).fetchall()
 
for r in rows:
    print(r)
conn.commit()

(1, '')
(2, '')
(3, 'Rear Admiral')
(4, '')
(5, '')
(6, '')
(7, '')
(8, '')
(9, '')
(10, '')


In [79]:
file = open('computerscientists.csv')
c = csv.reader(file)
next(c)
i = "INSERT or IGNORE INTO computerscientists(first,last,aka, id, birth,death) VALUES(?, ?,?,?,?,?)"
cursor.executemany(i, c)
select_all = "SELECT * FROM computerscientists"
rows = cursor.execute(select_all).fetchall()
for r in rows:
    print(r)
conn.commit()

(1, 'John', 'Backus', '', '1924-12-03', '2007-03-17')
(2, 'John', 'McCarthy', '', '1927-09-04', '2011-12-24')
(3, 'Grace', 'Hopper', '', '1906-12-09', '1992-01-01')
(4, 'Kristen', 'Nygaard', '', '1926-08-27', '2002-08-10')
(5, 'Ole-Johan', 'Dahl', '', '1931-10-12', '2002-06-29')
(6, 'Guido', 'van Rossum', '', '1956-01-31', '')
(7, 'Dennis', 'Ritchie', '', '1941-09-09', '2011-10-12')
(8, 'Yukihiro', 'Matsumoto', 'Matz', '1965-04-14', '')
(9, 'James', 'Gosling', '', '1955-05-19', '')
(10, 'Martin', 'Odersky', '', '', '')


In [80]:

file = open('contributions.csv')
c = csv.reader(file)
next(c)
n = "INSERT or IGNORE INTO  contributions(id, contribution) VALUES(?, ?)"
cursor.executemany(n, c)
select_all = "SELECT * FROM  contributions"
rows = cursor.execute(select_all).fetchall()

for r in rows:
    print(r)

conn.commit()

(1, 'Fortran')
(2, 'Lisp')
(3, 'UNIVAC')
(4, 'OOP')
(5, 'OOP')
(6, 'Python')
(7, 'UNIX')
(8, 'Ruby')
(9, 'Java')
(10, 'Scala')


In [81]:
file = open('awards.csv')

c = csv.reader(file)
next(c)

i = "INSERT or IGNORE into awards(id, award) VALUES(?, ?)"

cursor.executemany(i, c)

select_all = "SELECT * FROM awards"
rows = cursor.execute(select_all).fetchall()

for r in rows:
    print(r)


(1, "{'award': 'W.W. McDowell Award', 'year': 1967, 'by': 'IEEE Computer Society'}", None, None)
(2, "{'award': 'Turing Award', 'year': 1971, 'by': 'ACM'}", None, None)
(3, "{'award': 'Computer Sciences Man of the Year', 'year': 1969, 'by': 'Data Processing Management Association'}", None, None)
(4, "{'award': 'Rosing Prize', 'year': 1999, 'by': 'Norwegian Data Association'}", None, None)
(5, "{'award': 'Rosing Prize', 'year': 1999, 'by': 'Norwegian Data Association'}", None, None)
(6, "{'award': 'Award for the Advancement of Free Software', 'year': 2001, 'by': 'Free Software Foundation'}", None, None)
(7, "{'award': 'Turing Award', 'year': 1983, 'by': 'ACM'}", None, None)
(8, "{'award': 'Award for the Advancement of Free Software', 'year': '2011', 'by': 'Free Software Foundation'}", None, None)
(9, "{'award': 'The Economist Innovation Award', 'year': 2002, 'by': 'The Economist'}", None, None)


In [82]:
file = open('computerscientists.csv')

c = csv.reader(file)
next(c)

i = "INSERT or IGNORE INTO computerscientists(first,last,aka, id, birth,death) VALUES(?, ?,?,?,?,?)"

cursor.executemany(i, c)

select_all = "SELECT * FROM computerscientists"
rows = cursor.execute(select_all).fetchall()

for r in rows:
    print(r)

conn.commit

(1, 'John', 'Backus', '', '1924-12-03', '2007-03-17')
(2, 'John', 'McCarthy', '', '1927-09-04', '2011-12-24')
(3, 'Grace', 'Hopper', '', '1906-12-09', '1992-01-01')
(4, 'Kristen', 'Nygaard', '', '1926-08-27', '2002-08-10')
(5, 'Ole-Johan', 'Dahl', '', '1931-10-12', '2002-06-29')
(6, 'Guido', 'van Rossum', '', '1956-01-31', '')
(7, 'Dennis', 'Ritchie', '', '1941-09-09', '2011-10-12')
(8, 'Yukihiro', 'Matsumoto', 'Matz', '1965-04-14', '')
(9, 'James', 'Gosling', '', '1955-05-19', '')
(10, 'Martin', 'Odersky', '', '', '')


<function Connection.commit>

In [83]:
query='''
select (total_awards * 1.0)/total_scientists as avg_awards from
(select
(select count(*) from awards) as total_awards,
(select count(*) from computerscientists) as total_scientists)
'''
allrowsSelect(conn,query)

(0.9,)


In [84]:
#q2 What is the user id of the user that visited the site most frequently? 

# answer:userid 50
query='''
select distinct userid,count(*) As num_visit
from visits 
group by userid
order by  num_visit desc
limit 1


'''
allrowsSelect(conn,query)

(50, 363)


In [85]:
#q3

#answer: 9.52
query='''
select round(avg(price), 2)
from meals
where type = 'chinese';

'''
allrowsSelect(conn,query)

(9.52,)


In [86]:
#q4 What user bought the most meals?

# answer: 129
query='''
select userid, count(*) As num_purchase
from events
where event=='bought'
group by userid
order by num_purchase desc
limit 1





'''
allrowsSelect(conn,query)


(129, 101)


In [87]:
#q5 Quite a few users are tied for the most other users referred. For these top  referrers, what is the number of other users referred?

#Answer 6 times
query='''

select DISTINCT count(referred_by) AS num_Referred
From referrals
group BY referred_by

order BY num_Referred DESC
LIMIT 1;
'''
allrowsSelect(conn,query)

(6,)


In [88]:
#q6 What type of meal was most frequently shared by the user who bought the most  meals? 
# meals events inner join 


#answer: italian
query='''
select m.type,count(et.userid) as res
from meals m
inner join events et on et.meal_id=m.meal_id
where et.event=="share"
group by m.type
order by res
desc
limit 1

'''
allrowsSelect(conn,query)

('italian', 45289)


In [89]:
#q7 Using a window function, assign a quintile from 1 to 5 to each meal based on the  price with quintile 1 having the highest price and quintile 5 the lowest. What is  the lowest price in quintile 3?

#Answer: price 10
query='''

select min(price) from (
    select meal_id, price,
    ntile(5) over (order by price desc) percentile
    from meals
) a
where a.percentile = 3

'''
allrowsSelect(conn,query)

(10,)


In [90]:
#q8

#answer: chinese, french italian

query='''
drop view if exists next_meal
'''
execSQL(conn,query)

In [91]:
#q8

query='''
create view next_meal as
select userid, events.dt as current_date, m.type as current_meal_type,
 lead(m.type, 1) over (partition by userid order by events.dt) as next_meal
from events inner join meals m on events.meal_id = m.meal_id where event = 'bought';


'''
allrowsSelect(conn,query)

In [92]:
query='''
select current_meal_type from next_meal where userid = 2 limit 3;

'''
allrowsSelect(conn,query)

('chinese',)
('french',)
('italian',)


In [93]:
#q9 answer 4.29 less than the average

query='''


select min(diff) FROM (
SELECT meal_id,type,price, avg, (price-avg) AS diff 
FROM (SELECT meal_id, type, price, avg(price) OVER (PARTITION BY type) as avg
FROM meals ORDER BY meal_id) ) where type ='italian'

'''
allrowsSelect(conn,query)

(-4.292613636363637,)
